In [5]:
# Importer des modules
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import time
import os

1. ADPC POUR LES DATASET REDUITS 

In [25]:
def calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c):
    # Initialisation des variables
    num_time_steps = len(power_consumption_sequence)
    rho_values = np.zeros(num_time_steps)
    delta_values = np.zeros(num_time_steps)

    # Calcul des distances entre toutes les paires de valeurs de consommation d'énergie
    distance_matrix = cdist(power_consumption_sequence.reshape(-1, 1), power_consumption_sequence.reshape(-1, 1), metric='euclidean')
    
    # Calcul des valeurs rho
    for i in range(1, num_time_steps):
        for j in range(i + 1, num_time_steps):
            rho_values[i] += chi(distance_matrix[i, j] - d_c)
            rho_values[j] += chi(distance_matrix[i, j] - d_c)

    # Trier les indices par valeurs rho dans l'ordre décroissant
    sorted_indices = np.argsort(-rho_values)

    # Calcul des valeurs delta
    delta_values.fill(np.max(distance_matrix) * num_time_steps)
    delta_values[sorted_indices[0]] = -1
    for i in range(1, num_time_steps):
        for j in range(i):
            if distance_matrix[sorted_indices[i], sorted_indices[j]] < delta_values[sorted_indices[i]]:
                delta_values[sorted_indices[i]] = distance_matrix[sorted_indices[i], sorted_indices[j]]

    delta_values[sorted_indices[0]] = np.max(delta_values)

    # Déterminer la séquence de puissance correspondant aux différents états de fonctionnement de l'appareil
    power_sequence = [power_consumption_sequence[i] for i in range(num_time_steps) if rho_values[i] > rho_threshold and delta_values[i] > delta_threshold]
    num_working_states = len(power_sequence)

    return sorted(power_sequence), num_working_states  # Tri des valeurs avant de les retourner


def chi(x) :
    
    return x 



In [39]:
def adpc(power_consumption_sequence, rho_threshold, delta_threshold, d_c):
    # Initialisation des variables
    T = len(power_consumption_sequence)
    rho_values = np.zeros(T)
    delta_values = np.zeros(T)

    # Calcul des distances entre toutes les paires de valeurs de consommation d'énergie
    distance_matrix = cdist(power_consumption_sequence.reshape(-1, 1), power_consumption_sequence.reshape(-1, 1), metric='euclidean')
    
    # Calcul des valeurs rho
    for i in range(1, T-1):
        for j in range(i + 1, T):
            rho_values[i] += X_(distance_matrix[i, j] - d_c)
            rho_values[j] += X_(distance_matrix[i, j] - d_c)

    # Trier les indices par valeurs rho dans l'ordre décroissant
    sorted_indices = np.argsort(-rho_values)

    # Calcul des valeurs delta
    delta_values.fill(np.max(distance_matrix) * T)
    delta_values[sorted_indices[0]] = -1
    for i in range(1, T):
        for j in range(i):
            old_i, old_j = sorted_indices[i], sorted_indices[j]
            if distance_matrix[old_i, old_j] < delta_values[old_i]:
                delta_values[old_i] = distance_matrix[old_i, old_j]

    delta_values[sorted_indices[0]] = np.max(delta_values)

    # Déterminer la séquence de puissance correspondant aux différents états de fonctionnement de l'appareil
    power_sequence = [power_consumption_sequence[i] for i in range(T) if rho_values[i] > rho_threshold and delta_values[i] > delta_threshold]
    K = len(power_sequence)

    return sorted(power_sequence), K  # Tri des valeurs avant de les retourner


def X_(x) :
    
    if x < 0:
        return 1
    else:
        return 0


In [40]:
# Load simplified boiler data
file_name = 'boiler_226_15min.csv'
datasets_path = "./datasets/simplified/"
full_path = os.path.join(datasets_path, file_name)

boiler_data = pd.read_csv(full_path, index_col="timestamp")
print(boiler_data.head())
print("***")

# Extract the consumption sequence
power_consumption_sequence = boiler_data['power'].values

# Define parameters
rho_threshold = 0.01 * (len(power_consumption_sequence) - max(power_consumption_sequence))
delta_threshold = 0.1 * (max(power_consumption_sequence))
d_c = 5

# Run the ADPC algorithm
power_sequence, num_working_states = calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
print(power_sequence)
print(num_working_states)
print("---- ")
power_sequence, num_working_states = adpc(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
print(power_sequence)
print(num_working_states)

                           power
timestamp                       
2021-02-18 05:28:53+00:00    1.0
2021-02-18 05:43:53+00:00    1.0
2021-02-18 05:58:53+00:00    1.0
2021-02-18 06:13:53+00:00    1.0
2021-02-18 06:28:53+00:00    1.0
***
[176.0, 884.0, 1735.0, 2575.0]
4
---- 
[0.0]
1


In [28]:
# Load simplified dishwasher data
file_name = 'dishwasher_53_15min.csv'
datasets_path = "./datasets/simplified/"
full_path = os.path.join(datasets_path, file_name)

dishwasher_data = pd.read_csv(full_path, index_col="timestamp")
print(dishwasher_data.head())
print("****")

# Extract the consumption sequence
power_consumption_sequence = dishwasher_data['power'].values

# Define parameters
rho_threshold = 0.01 * (len(power_consumption_sequence) - max(power_consumption_sequence))
delta_threshold = 0.1 * (max(power_consumption_sequence))
d_c = 5

# Run the ADPC algorithm
power_sequence, num_working_states = calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
print(power_sequence)
print(num_working_states)
print("----")
power_sequence, num_working_states = adpc(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
print(power_sequence)
print(num_working_states)

                           power
timestamp                       
2021-01-18 00:51:25+00:00    2.0
2021-01-18 01:06:25+00:00    2.0
2021-01-18 01:21:25+00:00    3.0
2021-01-18 01:36:25+00:00    3.0
2021-01-18 01:51:25+00:00    2.0
****
[76.0, 1778.0]
2
----
[6.0, 1725.5]
2


In [30]:
# Load simplified computer data
file_name = 'computer_44_15min.csv'
datasets_path = "./datasets/simplified/"
full_path = os.path.join(datasets_path, file_name)

computer_data = pd.read_csv(full_path, index_col="timestamp")
print(computer_data.head())
print("****")

# Extract the consumption sequence
power_consumption_sequence = computer_data['power'].values

# Define parameters
rho_threshold = 0.01 * (len(power_consumption_sequence) - max(power_consumption_sequence))
delta_threshold = 0.1 * (max(power_consumption_sequence))
d_c = 5

# Run the ADPC algorithm
power_sequence, num_working_states = calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
print(power_sequence)
print(num_working_states)
print("---")
power_sequence, num_working_states = adpc(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
print(power_sequence)
print(num_working_states)

                           power
timestamp                       
2020-12-01 06:54:44+00:00    3.0
2020-12-01 07:09:44+00:00    3.0
2020-12-01 07:24:44+00:00    3.0
2020-12-01 07:39:44+00:00    3.0
2020-12-01 07:54:44+00:00    3.0
****
[2.0, 180.0, 264.0]
3
---
[2.0, 92.75, 139.5, 222.0]
4
